**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Start

Deployment and Service

https://kubernetes.io/docs/concepts/services-networking/connect-applications-service/

In [2]:
kubectl get pods -A

NAMESPACE     NAME                               READY   STATUS    RESTARTS      AGE
kube-system   coredns-78fcd69978-s857g           1/1     Running   0             20m
kube-system   etcd-minikube                      1/1     Running   0             20m
kube-system   kube-apiserver-minikube            1/1     Running   0             20m
kube-system   kube-controller-manager-minikube   1/1     Running   0             20m
kube-system   kube-proxy-phdq2                   1/1     Running   0             20m
kube-system   kube-scheduler-minikube            1/1     Running   0             20m
kube-system   storage-provisioner                1/1     Running   1 (20m ago)   20m


# Warning

**we install software from an unknown source**

In [3]:
kubectl create -f https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/service/networking/run-my-nginx.yaml

deployment.apps/my-nginx created


In [4]:
kubectl create -f https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/service/networking/nginx-svc.yaml

service/my-nginx created


In [15]:
kubectl get all

NAME                            READY   STATUS    RESTARTS   AGE
pod/my-nginx-5b56ccd65f-7br4l   1/1     Running   0          17m
pod/my-nginx-5b56ccd65f-h7w7q   1/1     Running   0          63s
pod/my-nginx-5b56ccd65f-lrbr4   1/1     Running   0          63s
pod/my-nginx-5b56ccd65f-nxbdq   1/1     Running   0          63s
pod/my-nginx-5b56ccd65f-p5k7h   1/1     Running   0          17m

NAME                 TYPE           CLUSTER-IP       EXTERNAL-IP      PORT(S)        AGE
service/kubernetes   ClusterIP      10.96.0.1        <none>           443/TCP        38m
service/my-nginx     LoadBalancer   10.105.208.150   10.105.208.150   80:32335/TCP   17m

NAME                       READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/my-nginx   5/5     5            5           17m

NAME                                  DESIRED   CURRENT   READY   AGE
replicaset.apps/my-nginx-5b56ccd65f   5         5         5       17m


In [ ]:
curl https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/service/networking/nginx-svc.yaml

In [ ]:
curl https://raw.githubusercontent.com/kubernetes/website/master/content/en/examples/service/networking/run-my-nginx.yaml

In [8]:
kubectl get service my-nginx

NAME       TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)   AGE
my-nginx   ClusterIP   10.105.208.150   <none>        80/TCP    2m3s


In [9]:
kubectl get service my-nginx -o yaml

apiVersion: v1
kind: Service
metadata:
  creationTimestamp: "2021-12-14T10:58:47Z"
  labels:
    run: my-nginx
  name: my-nginx
  namespace: default
  resourceVersion: "1347"
  uid: 5370558e-1212-4f5b-a270-0b0501e96aca
spec:
  clusterIP: 10.105.208.150
  clusterIPs:
  - 10.105.208.150
  internalTrafficPolicy: Cluster
  ipFamilies:
  - IPv4
  ipFamilyPolicy: SingleStack
  ports:
  - port: 80
    protocol: TCP
    targetPort: 80
  selector:
    run: my-nginx
  sessionAffinity: None
  type: ClusterIP
status:
  loadBalancer: {}


In [10]:
kubectl get service my-nginx -o jsonpath='{.spec.clusterIP}'

10.105.208.150

In [17]:
IP=$(kubectl get service my-nginx -o jsonpath='{.spec.clusterIP}')
curl $IP

<!DOCTYPE html>
<html>
<head>
<title>Welcome to nginx!</title>
<style>
html { color-scheme: light dark; }
body { width: 35em; margin: 0 auto;
font-family: Tahoma, Verdana, Arial, sans-serif; }
</style>
</head>
<body>
<h1>Welcome to nginx!</h1>
<p>If you see this page, the nginx web server is successfully installed and
working. Further configuration is required.</p>

<p>For online documentation and support please refer to
<a href="http://nginx.org/">nginx.org</a>.<br/>
Commercial support is available at
<a href="http://nginx.com/">nginx.com</a>.</p>

<p><em>Thank you for using nginx.</em></p>
</body>
</html>


In [16]:
echo http://$IP

http://10.105.208.150


In [13]:
kubectl get service -A -o jsonpath='{.items[*].spec.clusterIP}'

10.96.0.1 10.105.208.150 10.96.0.10

In [14]:
kubectl get pods --all-namespaces -o go-template \
--template="{{range .items}}{{.metadata.namespace}}/{{.metadata.name}}:{{println}}{{range .spec.containers}}  {{.image}}:{{.securityContext}}
{{end}}{{end}}" | grep --color=always -E '(^|privileged|NET_BIND_SERVICE)'


default/my-nginx-5b56ccd65f-7br4l:
  nginx:<no value>
default/my-nginx-5b56ccd65f-p5k7h:
  nginx:<no value>
kube-system/coredns-78fcd69978-s857g:
  k8s.gcr.io/coredns/coredns:v1.8.4:map[allowPrivilegeEscalation:false capabilities:map[add:[NET_BIND_SERVICE] drop:[all]] readOnlyRootFilesystem:true]
kube-system/etcd-minikube:
  k8s.gcr.io/etcd:3.5.0-0:<no value>
kube-system/kube-apiserver-minikube:
  k8s.gcr.io/kube-apiserver:v1.22.2:<no value>
kube-system/kube-controller-manager-minikube:
  k8s.gcr.io/kube-controller-manager:v1.22.2:<no value>
kube-system/kube-proxy-phdq2:
  k8s.gcr.io/kube-proxy:v1.22.2:map[privileged:true]
kube-system/kube-scheduler-minikube:
  k8s.gcr.io/kube-scheduler:v1.22.2:<no value>
kube-system/storage-provisioner:
  gcr.io/k8s-minikube/storage-provisioner:v5:<no value>
